In [1]:
# William Nourse
# Feb 11 2021

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import interact_manual
#%matplotlib widget

In [2]:
# Parameters
Cm = 10 # nF
Gm = 1 # uS
Er = -60 # mV
R = 20.0 # mV

dt = 0.01 # ms
tmax = 100 # ms
t = np.arange(0,tmax,dt)
numSteps = np.size(t)

In [3]:
def addition(Iapp1,Iapp2,k1,k2,Esyn=0,log=False):
    delEsyn = Esyn-Er
    gSyn1 = k1*R/(delEsyn-k1*R)
    gSyn2 = k2*R/(delEsyn-k2*R)
    
    if log:
        print('gSyn1 = %.3f, gSyn2 = %.3f'%(gSyn1,gSyn2))
    
    U1 = np.zeros(numSteps)
    U2 = np.zeros(numSteps)
    U3 = np.zeros(numSteps)
    
    for i in range(1,numSteps):
        U1[i] = U1[i-1] + dt/Cm*(Iapp1-Gm*U1[i-1])
        if U1[i-1]<R:
            g1 = U1[i-1]/R*gSyn1
        else:
            g1 = gSyn1
        U2[i] = U2[i-1] + dt/Cm*(Iapp2-Gm*U2[i-1])
        if U2[i-1]<R:
            g2 = U2[i-1]/R*gSyn2
        else:
            g2 = gSyn2
        U3[i] = U3[i-1] + dt/Cm*(g1*(delEsyn-U3[i-1])+g2*(delEsyn-U3[i-1])-Gm*U3[i-1])
        
    return U1,U2,U3

In [4]:
def interactAdd(Iapp1=5.0,Iapp2=5.0,k1=1.0,k2=1.0,Esyn=0):
    U1,U2,U3 = addition(Iapp1,Iapp2,k1,k2,Esyn=Esyn,log=True)
    
    plt.figure(figsize=(30,10))
    plt.axhline(y=(k1*Iapp1+k2*Iapp2),color='k',linestyle=':',label='Goal',linewidth=5)
    plt.plot(t,U1,linewidth=10,label='$U_1$')
    plt.plot(t,U2,linewidth=10,label='$U_2$')
    plt.plot(t,U3,linewidth=10,label='$U_3$')
    plt.legend()
    plt.xlabel('Time (ms)')
    plt.ylabel('Potential (mV)')
    plt.title('Response')
    
interact_manual(interactAdd,Iapp1=(0.0,R),Iapp2=(0.0,R),k1=(0.1,2.0),k2=(0.1,2.0),Esyn=(0.0,100))

interactive(children=(FloatSlider(value=5.0, description='Iapp1', max=20.0), FloatSlider(value=5.0, descriptio…

<function __main__.interactAdd(Iapp1=5.0, Iapp2=5.0, k1=1.0, k2=1.0, Esyn=0)>

# Takeaway: $U_3$ approximates $\sum k_i\cdot U_i$, and this approximation gets better as $\Delta E_{syn}$ gets larger

In [5]:
def interactAdd2D(k=0.5,Esyn=0):
    Iapp2 = R
    Iapp1 = np.linspace(0.0,R)
    U3str = np.zeros(np.size(Iapp1))
    k1 = k
    k2 = k
    for i in range(np.size(Iapp1)):
        _,_,U3 = addition(Iapp1[i],Iapp2,k1,k2,Esyn=Esyn,log=False)
        U3str[i] = U3[-1]

    plt.figure(figsize=(30,10))
    plt.subplot(1,2,1)
    plt.plot(Iapp1,U3str,linewidth=10,label='$U_3^*$')
    plt.plot(Iapp1,k1*Iapp1+k2*Iapp2,linewidth=10,linestyle=':',label='Goal')
    plt.xlabel('$I_{app,1}$')
    plt.ylabel('$U_3^*$')
    plt.title('Response')
    plt.legend()
    
    plt.subplot(1,2,2)
    plt.plot(Iapp1,U3str-(k1*Iapp1+k2*Iapp2),linewidth=10,label='$U_3^*$')
    plt.xlabel('$I_{app,1}$')
    plt.ylabel('Error')
    plt.title('Error as a function of $I_{app,1}$')

interact_manual(interactAdd2D,k=(0.0,2.0),Esyn=(0.0,100))

interactive(children=(FloatSlider(value=0.5, description='k', max=2.0), FloatSlider(value=0.0, description='Es…

<function __main__.interactAdd2D(k=0.5, Esyn=0)>

# Takeaway: As the desired sum increases, the behavior of an addition network diverges from the linear goal. The error decreases as $\Delta E_{syn}$ increases